In [0]:
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark=SparkSession.builder.appName("PROJECT").getOrCreate()

In [0]:
import logging
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark=SparkSession.builder.appName("POS_Daily_Transaction_File_Processing").getOrCreate()

In [0]:
# Function to check if mountpoint exists

def mount_exists(mount_point):
    mounts = [mount.mountPoint for mount in dbutils.fs.mounts()]
    return mount_point in mounts


# Define the mount point and storage details

mount_point="/mnt/pos_data_testing"
storage_account_name="pharmacydata2024"
container_name="test"
sas_token="DECBZEPbRnh8pmQGIHIRyH9QpredTuDf6nlHoQOn8pHqb1YBPzBHDrvo1uKIojLYh226lFaNS62B+AStdZxCCw=="


# Only create the mount if it doesn't already exist

if not mount_exists(mount_point):
    print(f"Mount point {mount_point} does not exist,creating mount.")

    dbutils.fs.mount(
        source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
        mount_point=mount_point,
        extra_configs={"fs.azure.account.key.pharmacydata2024.blob.core.windows.net":sas_token}
    )
    
    print(f"Mount point {mount_point} created successfully.")
else:
    print(f"Mount point {mount_point} already exists. No action taken.")


Mount point /mnt/pos_data_testing already exists. No action taken.


In [0]:
dbutils.fs.ls(mount_point)

[FileInfo(path='dbfs:/mnt/pos_data_testing/POS_Sales_Data_2024_demo_testing1.csv', name='POS_Sales_Data_2024_demo_testing1.csv', size=1054, modificationTime=1727337016000),
 FileInfo(path='dbfs:/mnt/pos_data_testing/test-date-processed-output/', name='test-date-processed-output/', size=0, modificationTime=0)]

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS postest

In [0]:
%sql
CREATE TABLE IF NOT EXISTS project_adb.postest.final_customer_kpi_test2 (
  CustomerID STRING,
  TotalSpending DOUBLE,
  TotalTransactions BIGINT,
  AvgSpendingPerTransaction DOUBLE,
  UniqueStores BIGINT,
  RetainedStatus BIGINT,
  RetentionRate DOUBLE,
  Jan2024 DOUBLE,
  Feb2024 DOUBLE,
  Mar2024 DOUBLE,
  Apr2024 DOUBLE,
  May2024 DOUBLE,
  Jun2024 DOUBLE,
  Jul2024 DOUBLE,
  Aug2024 DOUBLE,
  Sep2024 DOUBLE,
  Oct2024 DOUBLE,
  Nov2024 DOUBLE,
  Dec2024 DOUBLE,
  TopStoreLocation STRING,
  TopStoreSpending DOUBLE
) 
USING DELTA;


In [0]:
%sql
SELECT * FROM project_adb.postest.final_customer_kpi_test2;

CustomerID,TotalSpending,TotalTransactions,AvgSpendingPerTransaction,UniqueStores,RetainedStatus,RetentionRate,Jan2024,Feb2024,Mar2024,Apr2024,May2024,Jun2024,Jul2024,Aug2024,Sep2024,Oct2024,Nov2024,Dec2024,TopStoreLocation,TopStoreSpending


In [0]:
%sql
SELECT count(*) AS PREV_COUNT FROM project_adb.postest.final_customer_kpi_test2

PREV_COUNT
0


In [0]:
%sql
INSERT INTO project_adb.postest.final_customer_kpi_test2
VALUES 
('U021',85.89,1,1,85.89,0,0,NULL,NULL,NULL,NULL,NULL,85.89,NULL,NULL,NULL,NULL,NULL,NULL,'San Jose',85.89
),
('U097',124.91,1,1,124.91,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,124.91,NULL,NULL,NULL,NULL,'San Antonio',124.91
);


num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
SELECT * FROM project_adb.postest.final_customer_kpi_test2

CustomerID,TotalSpending,TotalTransactions,AvgSpendingPerTransaction,UniqueStores,RetainedStatus,RetentionRate,Jan2024,Feb2024,Mar2024,Apr2024,May2024,Jun2024,Jul2024,Aug2024,Sep2024,Oct2024,Nov2024,Dec2024,TopStoreLocation,TopStoreSpending
U021,85.89,1,1.0,85,0,0.0,null,null,null,null,null,85.89,null,null,null,null,null,null,San Jose,85.89
U097,124.91,1,1.0,124,0,0.0,null,null,null,null,null,null,null,124.91,null,null,null,null,San Antonio,124.91


In [0]:
pos_daily_file="/mnt/pos_data_testing/POS_Sales_Data_2024_demo_testing1.csv"
pos_daily_file_test=spark.read.format("csv").option("header","true").option("inferSchema","true").load(pos_daily_file)
display(pos_daily_file_test)

TransactionID,StoreID,StoreLocation,CashierID,CustomerID,CustomerAge,CustomerGender,ProductID,ProductCategory,ProductName,Quantity,UnitPrice,Timestamp,TotalAmount,DiscountApplied,TaxAmount,TotalAmountAfterTax
T00140,S001,San Jose,C002,U021,50,Female,P014,AIDS Treatment (Prescription),Diazepam,5,30.0,04-07-2024 02:33,150.0,50.0,10.0,110.0
T0015,S007,New York City,C003,U037,36,Male,P041,Cardiovascular Diseases (Prescription),Lisinopril,1,238.14,12-07-2024 00:16,238.14,29.24,16.6698,225.5698
T0016,S009,Chicago,C005,U009,41,Male,P005,Antibiotics (Prescription),Ciprofloxacin,4,121.95,23-07-2024 18:28,487.8,10.21,34.146,511.736
T0017,S008,Phoenix,C004,U039,58,Female,P010,Bone Health (OTC & Prescription),Cisplatin,2,94.82,10-06-2024 11:59,189.64,14.22,13.2748,188.6948
T0018,S010,San Jose,C005,U059,36,Female,P027,Eye Health (OTC & Prescription),Vitamin D,2,142.14,24-06-2024 07:49,284.28,37.58,19.8996,266.5996
T0019,S005,Philadelphia,C005,U065,53,Female,P008,Diabetes (Prescription & OTC),Olopatadine,4,203.3,19-07-2024 01:52,813.2,0.01,56.924,870.114


In [0]:
df_pos_daily_file=pos_daily_file_test.dropna()
display(df_pos_daily_file)

TransactionID,StoreID,StoreLocation,CashierID,CustomerID,CustomerAge,CustomerGender,ProductID,ProductCategory,ProductName,Quantity,UnitPrice,Timestamp,TotalAmount,DiscountApplied,TaxAmount,TotalAmountAfterTax
T00140,S001,San Jose,C002,U021,50,Female,P014,AIDS Treatment (Prescription),Diazepam,5,30.0,04-07-2024 02:33,150.0,50.0,10.0,110.0
T0015,S007,New York City,C003,U037,36,Male,P041,Cardiovascular Diseases (Prescription),Lisinopril,1,238.14,12-07-2024 00:16,238.14,29.24,16.6698,225.5698
T0016,S009,Chicago,C005,U009,41,Male,P005,Antibiotics (Prescription),Ciprofloxacin,4,121.95,23-07-2024 18:28,487.8,10.21,34.146,511.736
T0017,S008,Phoenix,C004,U039,58,Female,P010,Bone Health (OTC & Prescription),Cisplatin,2,94.82,10-06-2024 11:59,189.64,14.22,13.2748,188.6948
T0018,S010,San Jose,C005,U059,36,Female,P027,Eye Health (OTC & Prescription),Vitamin D,2,142.14,24-06-2024 07:49,284.28,37.58,19.8996,266.5996
T0019,S005,Philadelphia,C005,U065,53,Female,P008,Diabetes (Prescription & OTC),Olopatadine,4,203.3,19-07-2024 01:52,813.2,0.01,56.924,870.114


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

# Step 1: Convert Timestamp to Date and Extract Month
df_with_date = df_pos_daily_file.withColumn("Date", F.to_date(F.col("Timestamp"), "dd-MM-yyyy HH:mm")) \
                                .withColumn("Month", F.date_format(F.col("Date"), "MMM-yyyy"))  # Extract month in 'MMM-yyyy' format

# Step 2: Calculate Total Spending and Number of Transactions by Customer
customer_kpi = df_with_date.groupBy("CustomerID") \
    .agg(
        F.sum("TotalAmount").alias("TotalSpending"),
        F.count("TransactionID").alias("TotalTransactions"),
        F.countDistinct("StoreLocation").alias("UniqueStores"),
    )

# Step 3: Calculate Average Spending Per Transaction
customer_kpi = customer_kpi.withColumn("AvgSpendingPerTransaction", 
                                        F.col("TotalSpending") / F.col("TotalTransactions"))

# Step 4: Calculate Retained Customers
customer_purchases = df_with_date.groupBy("CustomerID") \
    .agg(F.count("TransactionID").alias("TotalPurchases"))

retained_customers = customer_purchases.groupBy("CustomerID") \
    .agg(F.count(F.when(F.col("TotalPurchases") > 1, 1)).alias("RetainedStatus"))

# Step 5: Join Retained Status with Customer KPI
customer_kpi = customer_kpi.join(retained_customers, "CustomerID", "left") \
    .fillna(0)

# Step 6: Calculate Retention Rate
customer_kpi = customer_kpi.withColumn("RetentionRate", 
                                        (F.col("RetainedStatus") / F.col("TotalTransactions")) * 100)

# Step 7: Identify Monthly Spending with Month Mapping
# Create month-year columns based on the extracted 'Month' from timestamp
monthly_spending = df_with_date.groupBy("CustomerID", "Month") \
    .agg(F.sum("TotalAmount").alias("MonthlySpending"))

# Step 8: Pivot Monthly Spending to Create Separate Columns with Custom Month Names
# Define month names based on the extracted 'Month'
monthly_spending_pivot = monthly_spending.groupBy("CustomerID") \
    .pivot("Month", ["Jan-2024", "Feb-2024", "Mar-2024", "Apr-2024", "May-2024", "Jun-2024", 
                     "Jul-2024", "Aug-2024", "Sep-2024", "Oct-2024", "Nov-2024", "Dec-2024"]) \
    .agg(F.sum("MonthlySpending")) \
    .withColumnRenamed("Jan-2024", "Jan2024") \
    .withColumnRenamed("Feb-2024", "Feb2024") \
    .withColumnRenamed("Mar-2024", "Mar2024") \
    .withColumnRenamed("Apr-2024", "Apr2024") \
    .withColumnRenamed("May-2024", "May2024") \
    .withColumnRenamed("Jun-2024", "Jun2024") \
    .withColumnRenamed("Jul-2024", "Jul2024") \
    .withColumnRenamed("Aug-2024", "Aug2024") \
    .withColumnRenamed("Sep-2024", "Sep2024") \
    .withColumnRenamed("Oct-2024", "Oct2024") \
    .withColumnRenamed("Nov-2024", "Nov2024") \
    .withColumnRenamed("Dec-2024", "Dec2024")

# Step 9: Join Pivoted Monthly Spending with Customer KPI
final_customer_kpi = customer_kpi.join(monthly_spending_pivot, "CustomerID", "left")

# Step 10: Identify Top Store for Each Customer Based on Spending
top_store = df_with_date.groupBy("CustomerID", "StoreLocation") \
    .agg(F.sum("TotalAmount").alias("StoreSpending")) \
    .withColumn("Rank", F.row_number().over(Window.partitionBy("CustomerID").orderBy(F.col("StoreSpending").desc()))) \
    .filter(F.col("Rank") == 1) \
    .select("CustomerID", "StoreLocation", "StoreSpending")

# Step 11: Final Join with Top Store
df_final_customer_kpi_test3 = final_customer_kpi.join(top_store, "CustomerID", "left") \
    .withColumnRenamed("StoreLocation", "TopStoreLocation") \
    .withColumnRenamed("StoreSpending", "TopStoreSpending") \
    .orderBy("TotalSpending", ascending=True)

# Show final KPI results with monthly sales columns like Jan2024, Feb2024, etc.
df_final_customer_kpi_test3.select("CustomerID", "TotalSpending", "TotalTransactions", 
                          "AvgSpendingPerTransaction", "UniqueStores", 
                          "RetainedStatus", "RetentionRate", 
                          "Jan2024", "Feb2024", "Mar2024", "Apr2024", 
                          "May2024", "Jun2024", "Jul2024", "Aug2024", 
                          "Sep2024", "Oct2024", "Nov2024", "Dec2024", 
                          "TopStoreLocation", "TopStoreSpending")


display(df_final_customer_kpi_test3)


CustomerID,TotalSpending,TotalTransactions,UniqueStores,AvgSpendingPerTransaction,RetainedStatus,RetentionRate,Jan2024,Feb2024,Mar2024,Apr2024,May2024,Jun2024,Jul2024,Aug2024,Sep2024,Oct2024,Nov2024,Dec2024,TopStoreLocation,TopStoreSpending
U021,150.0,1,1,150.0,0,0.0,null,null,null,null,null,null,150.0,null,null,null,null,null,San Jose,150.0
U039,189.64,1,1,189.64,0,0.0,null,null,null,null,null,189.64,null,null,null,null,null,null,Phoenix,189.64
U037,238.14,1,1,238.14,0,0.0,null,null,null,null,null,null,238.14,null,null,null,null,null,New York City,238.14
U059,284.28,1,1,284.28,0,0.0,null,null,null,null,null,284.28,null,null,null,null,null,null,San Jose,284.28
U009,487.8,1,1,487.8,0,0.0,null,null,null,null,null,null,487.8,null,null,null,null,null,Chicago,487.8
U065,813.2,1,1,813.2,0,0.0,null,null,null,null,null,null,813.2,null,null,null,null,null,Philadelphia,813.2


In [0]:
prev_cust_data_kpi_df_test2=spark.table("project_adb.postest.final_customer_kpi_test2")
display(prev_cust_data_kpi_df_test2)

CustomerID,TotalSpending,TotalTransactions,AvgSpendingPerTransaction,UniqueStores,RetainedStatus,RetentionRate,Jan2024,Feb2024,Mar2024,Apr2024,May2024,Jun2024,Jul2024,Aug2024,Sep2024,Oct2024,Nov2024,Dec2024,TopStoreLocation,TopStoreSpending
U021,85.89,1,1.0,85,0,0.0,null,null,null,null,null,85.89,null,null,null,null,null,null,San Jose,85.89
U097,124.91,1,1.0,124,0,0.0,null,null,null,null,null,null,null,124.91,null,null,null,null,San Antonio,124.91


In [0]:
from delta.tables import DeltaTable

In [0]:
# Load the existing non-Delta table into a DataFrame
df_non_delta2 = spark.table("project_adb.postest.final_customer_kpi_test2")

# Overwrite the existing table with Delta format
df_non_delta2.write.format("delta").mode("overwrite").saveAsTable("project_adb.postest.final_customer_kpi_test2")


In [0]:
target_delta_table=DeltaTable.forName(spark,"project_adb.postest.final_customer_kpi_test2")

# Perform the merge operation
target_delta_table.alias("target").merge(
    source=df_final_customer_kpi_test3.alias("source"),
    condition="target.CustomerID = source.CustomerID"
).whenMatchedUpdate(
    set={
        "TotalSpending": "target.TotalSpending + source.TotalSpending",
        "TotalTransactions": "target.TotalTransactions + source.TotalTransactions",
        "AvgSpendingPerTransaction": "(target.TotalSpending + source.TotalSpending) / (target.TotalTransactions + source.TotalTransactions)",
        "UniqueStores": "source.UniqueStores",
        "RetainedStatus": "source.RetainedStatus",
        "RetentionRate": "source.RetentionRate",
        "TopStoreLocation": "source.TopStoreLocation",
        "TopStoreSpending": "source.TopStoreSpending",
        # Update monthly spending columns
        "Jan2024": "COALESCE(target.Jan2024, 0) + COALESCE(source.Jan2024, 0)",
        "Feb2024": "COALESCE(target.Feb2024, 0) + COALESCE(source.Feb2024, 0)",
        "Mar2024": "COALESCE(target.Mar2024, 0) + COALESCE(source.Mar2024, 0)",
        "Apr2024": "COALESCE(target.Apr2024, 0) + COALESCE(source.Apr2024, 0)",
        "May2024": "COALESCE(target.May2024, 0) + COALESCE(source.May2024, 0)",
        "Jun2024": "COALESCE(target.Jun2024, 0) + COALESCE(source.Jun2024, 0)",
        "Jul2024": "COALESCE(target.Jul2024, 0) + COALESCE(source.Jul2024, 0)",
        "Aug2024": "COALESCE(target.Aug2024, 0) + COALESCE(source.Aug2024, 0)",
        "Sep2024": "COALESCE(target.Sep2024, 0) + COALESCE(source.Sep2024, 0)",
        "Oct2024": "COALESCE(target.Oct2024, 0) + COALESCE(source.Oct2024, 0)",
        "Nov2024": "COALESCE(target.Nov2024, 0) + COALESCE(source.Nov2024, 0)",
        "Dec2024": "COALESCE(target.Dec2024, 0) + COALESCE(source.Dec2024, 0)",
    }
).whenNotMatchedInsert(
    values={
        "CustomerID": "source.CustomerID",
        "TotalSpending": "source.TotalSpending",
        "TotalTransactions": "source.TotalTransactions",
        "AvgSpendingPerTransaction": "source.AvgSpendingPerTransaction",
        "UniqueStores": "source.UniqueStores",
        "RetainedStatus": "source.RetainedStatus",
        "RetentionRate": "source.RetentionRate",
        "TopStoreLocation": "source.TopStoreLocation",
        "TopStoreSpending": "source.TopStoreSpending",
        # Insert monthly spending columns
        "Jan2024": "source.Jan2024",
        "Feb2024": "source.Feb2024",
        "Mar2024": "source.Mar2024",
        "Apr2024": "source.Apr2024",
        "May2024": "source.May2024",
        "Jun2024": "source.Jun2024",
        "Jul2024": "source.Jul2024",
        "Aug2024": "source.Aug2024",
        "Sep2024": "source.Sep2024",
        "Oct2024": "source.Oct2024",
        "Nov2024": "source.Nov2024",
        "Dec2024": "source.Dec2024",
    }
).execute()


In [0]:
%sql
SELECT * FROM project_adb.postest.final_customer_kpi_test2

CustomerID,TotalSpending,TotalTransactions,AvgSpendingPerTransaction,UniqueStores,RetainedStatus,RetentionRate,Jan2024,Feb2024,Mar2024,Apr2024,May2024,Jun2024,Jul2024,Aug2024,Sep2024,Oct2024,Nov2024,Dec2024,TopStoreLocation,TopStoreSpending
U021,235.89,2,117.945,1,0,0.0,0.0,0.0,0.0,0.0,0.0,85.89,150.0,0.0,0.0,0.0,0.0,0.0,San Jose,150.0
U039,189.64,1,189.64,1,0,0.0,null,null,null,null,null,189.64,null,null,null,null,null,null,Phoenix,189.64
U037,238.14,1,238.14,1,0,0.0,null,null,null,null,null,null,238.14,null,null,null,null,null,New York City,238.14
U059,284.28,1,284.28,1,0,0.0,null,null,null,null,null,284.28,null,null,null,null,null,null,San Jose,284.28
U009,487.8,1,487.8,1,0,0.0,null,null,null,null,null,null,487.8,null,null,null,null,null,Chicago,487.8
U065,813.2,1,813.2,1,0,0.0,null,null,null,null,null,null,813.2,null,null,null,null,null,Philadelphia,813.2
U097,124.91,1,1.0,124,0,0.0,null,null,null,null,null,null,null,124.91,null,null,null,null,San Antonio,124.91


In [0]:
%sql
SELECT count(*) AS UPDATED_COUNT FROM project_adb.postest.final_customer_kpi_test2

UPDATED_COUNT
7


In [0]:
#Save output file in different file formats with name like 'part-00000' in adls

df2=spark.table("project_adb.postest.final_customer_kpi_test2")
df2.write.mode("overwrite").format("csv").save("/mnt/pos_data_testing/df_customer_filtered.csv",header=True)

In [0]:
from pyspark.sql.functions import current_date

# Get the current date in 'yyyy-MM-dd' format
current_date_value = spark.sql("SELECT date_format(current_date(), 'yyyy-MM-dd')").collect()[0][0]
print(current_date_value)

# Create the directory for today's date
dbutils.fs.mkdirs(f"dbfs:/mnt/pos_data_testing/test-date-processed-output-cust/{current_date_value}")

# Define the destination path for saving the DataFrame
destination_path_csv = f"dbfs:/mnt/pos_data_testing/test-date-processed-output-cust/{current_date_value}/output_file"

# Write the DataFrame to CSV
df2.write.mode("overwrite").format("csv").save(destination_path_csv, header=True)

# List files in the source directory
files = dbutils.fs.ls("dbfs:/mnt/dss-project-works/project-data/test_output_from_adb/delta-table-test-output/")

# Define custom file name for the copied file
custom_file_name = f"customer_kpi_{current_date_value}.csv"

# Copy the relevant file(s) to the destination directory with the custom name
for file in files:
    if "part-00000-tid-" in file.name:
        # Copy the file to the destination with the custom name
        dbutils.fs.cp(file.path, f"dbfs:/mnt/pos_data_testing/test-date-processed-output-cust/{current_date_value}/{custom_file_name}")
        break

# Now, move the entire output directory to a new name
new_directory_path = f"dbfs:/mnt/pos_data_testing/test-date-processed-output/{current_date_value}/output_file_temp"
dbutils.fs.mv(destination_path_csv, new_directory_path, recurse=True)

# Rename part files within the new directory to the custom file name
part_files = dbutils.fs.ls(new_directory_path)

# Move part files to the new custom name
for part_file in part_files:
    if part_file.name.startswith("part-"):
        dbutils.fs.mv(part_file.path, f"dbfs:/mnt/pos_data_testing/test-date-processed-output/{current_date_value}/{custom_file_name}",recurse=True)


2024-09-26


In [0]:
# %sql
# TRUNCATE TABLE project_adb.postest.final_customer_kpi_test2

In [0]:

# ------------`recurse=True` ---------
# option in Databricks (and in many file system operations) allows you to perform actions on a directory and all its contents, including subdirectories and files, recursively.


# 1. **Moving Directories**: When you move a directory, setting `recurse=True` ensures that all files and subdirectories within that directory are also moved to the new location. If `recurse` is not set to `True`, the operation will fail if the source is a directory with contents.

# 2. **Deleting Directories**: When deleting a directory, `recurse=True` allows the deletion of the directory along with all its contents, rather than just the directory itself.

# 3. **Copying Directories**: Similarly, when copying, it allows you to copy the entire directory structure, including all nested files and subdirectories.

# ### Example:
# - **Without `recurse=True`**:
#   - If you try to move a directory without `recurse=True`, and that directory contains files or subdirectories, you’ll get an error saying that you cannot move a non-empty directory.
  
# - **With `recurse=True`**:
#   - It allows the move to proceed, transferring all files and subdirectories to the new location.

# In summary, `recurse=True` is crucial when you want to manipulate directories that contain multiple files and subdirectories, ensuring that your operations encompass everything within the specified directory.
# '''